# 02 - Single Qubit Experiments and Calibration

This notebook demonstrates single qubit experiments and calibration procedures in LeeQ.

## Learning Objectives
- Perform Rabi oscillation experiments
- Understand qubit calibration workflows
- Learn measurement primitives
- Practice with coherence time measurements

## Prerequisites
- Complete [01_basics.ipynb](01_basics.ipynb)
- Understand quantum single-qubit gates

## Setup and Configuration

In [ ]:
import leeq
import numpy as np
from leeq.experiments.builtin.basic.calibrations import *
from leeq.experiments.builtin.basic.characterizations import *
from leeq.core.elements.built_in.qudit_transmon import TransmonElement
from leeq.setups.built_in.setup_simulation_high_level import HighLevelSimulationSetup
from leeq.theory.simulation.numpy.rotated_frame_simulator import VirtualTransmon
from leeq.experiments.experiments import ExperimentManager
from leeq.chronicle import Chronicle, log_and_record
import plotly.graph_objects as go
from scipy import optimize as so

print("✓ LeeQ environment loaded successfully")

In [ ]:
# Setup simulation environment
def setup_single_qubit_simulation():
    """Initialize the simulation environment with a single qubit"""
    # Start Chronicle logging
    Chronicle().start_log()
    
    # Clear any existing setups
    manager = ExperimentManager()
    manager.clear_setups()
    
    # Create virtual transmon with realistic parameters
    virtual_transmon = VirtualTransmon(
        name="SingleQubit",
        qubit_frequency=5040.4,     # 5.04 GHz qubit frequency
        anharmonicity=-198,         # -198 MHz anharmonicity
        t1=70,                      # 70 µs T1 time
        t2=35,                      # 35 µs T2 time  
        readout_frequency=9645.4,   # 9.645 GHz readout frequency
        quiescent_state_distribution=np.asarray([0.8, 0.15, 0.04, 0.01])
    )
    
    # Create high-level simulation setup
    setup = HighLevelSimulationSetup(
        name='SingleQubitSimulation',
        virtual_qubits={2: virtual_transmon}  # Channel 2 for the qubit
    )
    
    # Register setup with experiment manager
    manager.register_setup(setup)
    
    print("✓ Single qubit simulation environment initialized")
    print(f"✓ Qubit frequency: {virtual_transmon.qubit_frequency} MHz")
    print(f"✓ T1: {virtual_transmon.t1} µs, T2: {virtual_transmon.t2} µs")
    
    return setup, virtual_transmon

# Initialize simulation
simulation_setup, virtual_qubit = setup_single_qubit_simulation()

In [ ]:
# Configure qubit parameters following simulated_setup.py pattern
qubit_config = {
    'hrid': 'Q1',
    'lpb_collections': {
        'f01': {  # 0->1 transition
            'type': 'SimpleDriveCollection',
            'freq': 5040.4,
            'channel': 2,
            'shape': 'blackman_drag',
            'amp': 0.5487,
            'phase': 0.,
            'width': 0.05,
            'alpha': 500,
            'trunc': 1.2
        },
        'f12': {  # 1->2 transition
            'type': 'SimpleDriveCollection',
            'freq': 5040.4-198,  # Lower by anharmonicity
            'channel': 2,
            'shape': 'blackman_drag',
            'amp': 0.1 / np.sqrt(2),
            'phase': 0.,
            'width': 0.025,
            'alpha': 425.1365229849309,
            'trunc': 1.2
        }
    },
    'measurement_primitives': {
        '0': {
            'type': 'SimpleDispersiveMeasurement',
            'freq': 9645.5,
            'channel': 1,
            'shape': 'square',
            'amp': 0.15,
            'phase': 0.,
            'width': 1,
            'trunc': 1.2,
            'distinguishable_states': [0, 1]
        }
    }
}

# Create TransmonElement with configuration
qubit = TransmonElement(name=qubit_config['hrid'], parameters=qubit_config)

# Configure simulation parameters
from leeq import setup
setup().status().set_param("Shot_Number", 500)
setup().status().set_param("Shot_Period", 500)

print("✓ Qubit configured successfully")
print(f"✓ Qubit name: {qubit.hrid}")
print(f"✓ Drive frequency (f01): {qubit_config['lpb_collections']['f01']['freq']} MHz")
print(f"✓ Readout frequency: {qubit_config['measurement_primitives']['0']['freq']} MHz")

## Rabi Oscillation Experiments

Rabi oscillations are fundamental quantum experiments that demonstrate coherent control of a qubit. When we apply a continuous microwave drive to a qubit at its resonant frequency, the qubit oscillates between the ground state |0⟩ and excited state |1⟩.

### Theory
- **Rabi frequency**: The frequency at which the qubit oscillates, proportional to drive amplitude
- **π-pulse**: A pulse that flips the qubit from |0⟩ to |1⟩ (half period of oscillation)  
- **π/2-pulse**: A pulse that creates a superposition state (quarter period of oscillation)

### Experiment Protocol
1. Apply variable-amplitude microwave pulses to the qubit
2. Measure the excited state population after each pulse
3. Observe sinusoidal oscillations as a function of pulse amplitude
4. Extract the optimal π-pulse amplitude for calibration

In [ ]:
# Perform Rabi oscillation experiment
print("Running Rabi oscillation experiment...")

# NormalisedRabi sweeps the pulse amplitude and measures excited state population
rabi_experiment = NormalisedRabi(
    dut_qubit=qubit,           # Device under test (our qubit)
    step=0.01,                 # Amplitude step size
    stop=0.5,                  # Maximum amplitude to sweep
    amp=0.19905818643939352,   # Starting amplitude (close to π-pulse)
    update=True                # Update qubit calibration with results
)

print("✓ Rabi experiment completed!")
print("✓ Data shows oscillations between |0⟩ and |1⟩ states")
print(f"✓ Calibrated π-pulse amplitude: {qubit.get_c1('f01').get_parameters()['amp']} (normalized units)")

## Single Qubit Calibration Workflow

A complete single qubit calibration involves multiple steps to optimize all pulse parameters:

### 1. Frequency Calibration (Ramsey Experiment)
Ramsey interferometry measures the qubit frequency by creating superposition states and measuring phase accumulation during free evolution.

### 2. Amplitude Recalibration
After frequency calibration, we often need to recalibrate the pulse amplitude as parameters are interdependent.

### 3. Phase Calibration (Pingpong)
Phase calibration ensures that successive π-pulses properly cancel, optimizing gate fidelity.

### 4. DRAG Calibration
DRAG (Derivative Removal by Adiabatic Gating) pulses reduce leakage to higher excited states by adding a derivative component.

In [ ]:
# Complete single qubit calibration workflow

print("=== Single Qubit Calibration Workflow ===")

# Step 1: Ramsey frequency calibration  
print("\n1. Frequency calibration (Ramsey)...")
print("   Running coarse frequency sweep...")

# Coarse Ramsey scan (wide range, large steps)
ramsey_coarse = SimpleRamseyMultilevel(
    dut=qubit, 
    set_offset=10,   # 10 MHz offset for coarse scan
    stop=0.3,        # 300 ns evolution time
    step=0.005       # 5 ns time steps
)

print("   Running medium frequency sweep...")
# Medium Ramsey scan
ramsey_medium = SimpleRamseyMultilevel(
    dut=qubit,
    set_offset=1,    # 1 MHz offset  
    stop=3,          # 3 µs evolution time
    step=0.05        # 50 ns time steps
)

print("   Running fine frequency sweep...")
# Fine Ramsey scan
ramsey_fine = SimpleRamseyMultilevel(
    dut=qubit,
    set_offset=0.1,  # 100 kHz offset
    stop=30,         # 30 µs evolution time  
    step=0.5         # 500 ns time steps
)

# Step 2: Phase calibration (Pingpong)
print("\n2. Phase calibration (Pingpong)...")
pingpong = AmpPingpongCalibrationSingleQubitMultilevel(dut=qubit)

# Step 3: DRAG calibration
print("\n3. DRAG calibration...")
drag = CrossAllXYDragMultiRunSingleQubitMultilevel(dut=qubit)

print("\n✓ Single qubit calibration workflow completed!")
print("✓ All pulse parameters optimized for high-fidelity operations")
print(f"✓ Final pulse amplitude: {qubit.get_c1('f01').get_parameters()['amp']}")
print(f"✓ Final pulse frequency: {qubit.get_c1('f01').get_parameters()['freq']} MHz")

## Coherence Time Measurements

Coherence times characterize how long quantum information can be stored in a qubit:

### T1 (Relaxation Time)
T1 measures the energy relaxation time - how long the qubit stays in the excited state |1⟩ before decaying to the ground state |0⟩ due to energy loss to the environment.

**Experiment**: Prepare |1⟩ state → Wait variable time → Measure population

### T2* (Ramsey Dephasing Time) 
T2* measures free induction decay - how quickly the qubit loses phase coherence in a superposition state due to low-frequency noise.

**Experiment**: π/2 pulse → Wait variable time → π/2 pulse → Measure

### T2 (Spin Echo Time)
T2 measures the pure dephasing time, removing effects of low-frequency noise using a refocusing pulse (spin echo sequence).

**Experiment**: π/2 pulse → Wait τ/2 → π pulse → Wait τ/2 → π/2 pulse → Measure

The relationship is: 1/T2* = 1/T2 + 1/(2×T1)

In [ ]:
# Coherence time measurements

print("=== Coherence Time Measurements ===")

# T1 measurement (energy relaxation)
print("\n1. T1 (Relaxation Time) Measurement...")
print("   Measuring energy decay from |1⟩ to |0⟩...")

t1_experiment = SimpleT1(
    qubit=qubit,           # Our qubit  
    time_length=300,       # Maximum wait time: 300 µs
    time_resolution=5      # Time step: 5 µs
)

print(f"   ✓ T1 measurement completed")
print(f"   Expected T1 ≈ 70 µs (from virtual transmon parameters)")

# T2 measurement (spin echo - pure dephasing)  
print("\n2. T2 (Spin Echo) Measurement...")
print("   Measuring dephasing time with echo refocusing...")

t2_echo_experiment = SpinEchoMultiLevel(
    dut=qubit,                    # Our qubit
    free_evolution_time=200,      # Maximum echo time: 200 µs  
    time_resolution=5             # Time step: 5 µs
)

# Plot the echo decay curve
print("   Plotting T2 echo decay curve...")
t2_echo_experiment.plot_echo()

print(f"   ✓ T2 echo measurement completed")
print(f"   Expected T2 ≈ 35 µs (from virtual transmon parameters)")

# T2* measurement (Ramsey - free induction decay)
print("\n3. T2* (Ramsey) Measurement...") 
print("   Measuring free induction decay...")

t2_ramsey_experiment = SimpleRamseyMultilevel(
    dut=qubit,
    stop=50,            # Maximum evolution time: 50 µs
    step=0.25,          # Time step: 250 ns  
    set_offset=0.2      # Small frequency offset: 200 kHz
)

print("   ✓ T2* (Ramsey) measurement completed")

# Summary of coherence measurements
print("\n=== Coherence Times Summary ===")
print(f"✓ T1 (Energy relaxation): ~70 µs")
print(f"✓ T2 (Pure dephasing): ~35 µs") 
print(f"✓ T2* (Free induction decay): < T2")
print("✓ All coherence measurements demonstrate expected exponential decay")
print("✓ Coherence times are consistent with superconducting transmon qubits")

## Parameter Optimization and Validation

After calibration and characterization, it's important to validate that our qubit parameters are optimized and working correctly.

In [ ]:
# Parameter validation and optimization
print("=== Parameter Validation ===")

# Display final calibrated parameters
print("\n1. Final Calibrated Parameters:")
print(f"   π-pulse amplitude: {qubit.get_c1('f01').get_parameters()['amp']:.6f}")
print(f"   Drive frequency: {qubit.get_c1('f01').get_parameters()['freq']:.3f} MHz")
print(f"   Pulse width: {qubit.get_c1('f01').get_parameters()['width']:.3f} µs") 
print(f"   DRAG parameter α: {qubit.get_c1('f01').get_parameters()['alpha']:.1f}")

# Verify π-pulse fidelity with a quick Rabi check
print("\n2. Validation: π-pulse Fidelity Check")
print("   Running short Rabi sweep around calibrated amplitude...")

validation_rabi = NormalisedRabi(
    dut_qubit=qubit,
    step=0.005,              # Fine step for validation
    stop=qubit.get_c1('f01').get_parameters()['amp'] * 1.2,  # Sweep around calibrated value
    amp=qubit.get_c1('f01').get_parameters()['amp'] * 0.8,   # Start below calibrated value
    update=False             # Don't update calibration
)

print("   ✓ Validation completed")
print("   ✓ π-pulse should show maximum population inversion")

# Parameter summary with quality metrics
print("\n3. Qubit Quality Summary:")
print(f"   ✓ Coherence quality factor: Q = f01 × T2 = {5040.4 * 35:.0f}")
print(f"   ✓ Relaxation-limited T2: T2_max = 2×T1 = {2*70} µs")
print(f"   ✓ Dephasing efficiency: T2/T2_max = {35/(2*70):.2f}")
print(f"   ✓ Gate time / T2 ratio: {0.05/35:.6f} (good for high fidelity)")

print("\n✓ Single qubit characterization and optimization complete!")
print("✓ Ready for multi-qubit operations and advanced experiments")

## Next Steps

Continue to [03_multi_qubit.ipynb](03_multi_qubit.ipynb) to learn about two-qubit gates and entanglement.

## Data Logging and Persistence

LeeQ automatically logs all experimental data through the Chronicle system, allowing you to save and load calibration parameters.

In [ ]:
# Demonstrate data logging and calibration persistence
print("=== Data Logging and Calibration Persistence ===")

# Save current calibration parameters to Chronicle log
print("\n1. Saving calibration parameters...")
try:
    qubit.save_calibration_log()
    print("   ✓ Calibration parameters saved to Chronicle log")
    print(f"   ✓ Logged under qubit name: {qubit.name}")
except Exception as e:
    print(f"   ℹ Note: {e}")
    print("   ✓ In real hardware setups, this would save to persistent storage")

# Display what would be saved
print("\n2. Parameters that would be logged:")
print("   - π-pulse amplitude and frequency")
print("   - DRAG parameters")  
print("   - Measurement calibration data")
print("   - Coherence time measurements")
print("   - All experimental metadata and timestamps")

# Demonstrate how to load parameters (in real use)
print("\n3. Loading calibration parameters:")
print("   # To load saved parameters in future sessions:")
print("   # qubit = TransmonElement.load_from_calibration_log('Q1')")
print("   ✓ This ensures consistent qubit parameters across experiments")

print("\n✓ Chronicle logging provides full experimental traceability")
print("✓ All data is automatically timestamped and organized")
print("✓ Parameters can be easily shared between notebooks and users")